In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import mytools
import re

In [3]:
data_excel = pd.ExcelFile('G:/1-规划/崔开通崔可研规划/2020年任务清单/20200924/2020年任务清单（场景+MR）0917YC-打标0924.xlsx')

In [4]:
data = data_excel.parse(sheet_name=data_excel.sheet_names[0])

In [5]:
data['TDL规划站号（新）'] = data['TDL规划站号（新）'].fillna('无')

In [6]:
qingdan_yuan = data.loc[data['分类']=='2020年5月新任务清单']

# -------全网进度--------
## 优化进度

In [7]:
qingdan = qingdan_yuan.copy()

In [8]:
# 优化总点youhua_z，优化完成youhua_w，
youhua_z = qingdan.loc[(qingdan['优化']=='优化')]['唯一编号'].count()
youhua_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]['唯一编号'].count()
youhua_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)& (qingdan['优化完成']==1)]['唯一编号'].count()

In [9]:
#youhua已经df，优化完成df
youhua_data_z = qingdan.loc[(qingdan['优化']=='优化') ]
youhua_data_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]
youhua_data_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1) & (qingdan['优化完成']==1)]

In [10]:
#地市的完成数量y_w，地市的派发总数量y_z
y_z = youhua_data_z.groupby(by = '地市').agg({'唯一编号':'count'})
y_w = youhua_data_w.groupby(by = '地市').agg({'唯一编号':'count'})
y_p = youhua_data_p.groupby(by = '地市').agg({'唯一编号':'count'})

In [11]:
# 统计每个地市的派发和完成
youhuajindu=pd.concat([y_z,y_p,y_w],axis = 1)
youhuajindu.columns = ['总量','派发','完成量']
youhuajindu['完成派发进度'] = youhuajindu['完成量']/youhuajindu['派发']
youhuajindu['整体进度'] = youhuajindu['完成量']/youhuajindu['总量']

In [12]:
#按照格式输出
youhua_wanchengdugao=list(youhuajindu.sort_values(by="完成派发进度",ascending= False).head(3).index)  #排序 升序True 降序False by列

In [13]:
'MR竞对优化工单总计'+str(youhua_z)+'单。已派发'+str(youhua_p)+'已完成'+str(youhua_w)+ '解决率'+str(youhua_w/youhua_p)+ \
str(youhua_wanchengdugao[0])+'、'+str(youhua_wanchengdugao[1])+'、'+str(youhua_wanchengdugao[2])+'派发工单解决率相对较高'

'MR竞对优化工单总计4313单。已派发1561已完成965解决率0.618193465727098荆州、随州、天门派发工单解决率相对较高'

In [14]:
youhuajindu.sort_values(by="完成派发进度",ascending= False)

,总量,派发,完成量,完成派发进度,整体进度
地市,,,,,
荆州,197,133,104,0.781955,0.527919
随州,115,63,49,0.777778,0.426087
天门,42,42,32,0.761905,0.761905
襄阳,217,90,67,0.744444,0.308756
恩施,633,191,129,0.675393,0.203791
荆门,165,66,44,0.666667,0.266667
武汉,1077,331,218,0.658610,0.202414
潜江,34,28,18,0.642857,0.529412
孝感,302,114,70,0.614035,0.231788


# 催开进度

In [15]:
qingdan = qingdan_yuan.copy()

In [16]:
# cuikai的数量统计
cuikai_zdanzi = qingdan.loc[(qingdan['催开']=='催开') ]['唯一编号'].count()
cuikai_z = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
cuikai_w = qingdan.loc[(
    qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [17]:
#催开总量和完成的df
cuikai_quchong = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') 
cuikai_quchong_wancheng = qingdan.loc[(
    qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') 

In [18]:
#个地市的进度
cuikai_zongliang = cuikai_quchong.groupby(by = '地市').agg({'唯一编号':'count'})
cuikai_wanchengliang = cuikai_quchong_wancheng.groupby(by = '地市').agg({'唯一编号':'count'})
cuikaijindu=pd.concat([cuikai_zongliang,cuikai_wanchengliang],axis = 1)
cuikaijindu.columns = ['总量','完成量']
cuikaijindu['进度'] = cuikaijindu['完成量']/cuikaijindu['总量']

In [19]:
cuikai_wanchengdudi=list(cuikaijindu.sort_values(by="进度",ascending= True).head(1).index)  #排序 升序True 降序False by列
a=cuikai_wanchengdudi[0]
jindudi=cuikai_quchong.loc[(cuikai_quchong['地市']==a) & (cuikai_quchong['催开完成']!=1)
                          ].groupby(by='工期核实').agg({'唯一编号':'count'}).sort_values(by="唯一编号",ascending= False).head(1).index
'催开工单共'+str(cuikai_zdanzi)+'单，涉及逻辑站\
点'+str(cuikai_z)+'个，已开通站点'+str(cuikai_w)+'个(本月新增201个），催开完成率\
为'+str(cuikai_w/cuikai_z*100)[:5]+'%'+'。'+cuikai_wanchengdudi[0]+'地市整体解决进度较慢，主要为'+jindudi[0]+'未开通占比较\
高，需加快站点建设进度。'+'已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

'催开工单共2827单，涉及逻辑站点1760个，已开通站点1047个(本月新增201个），催开完成率为59.48%。天门地市整体解决进度较慢，主要为5月前完成-农村未开通占比较高，需加快站点建设进度。已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

In [20]:
cuikaijindu.sort_values(by="进度",ascending= True)

,总量,完成量,进度
地市,,,
天门,27,6,0.222222
咸宁,169,51,0.301775
恩施,129,41,0.317829
潜江,8,3,0.375000
荆门,55,22,0.400000
鄂州,18,8,0.444444
黄石,97,45,0.463918
荆州,102,49,0.480392
黄冈,135,75,0.555556


In [21]:
cuikai_quchong.to_clipboard()

# 可研进度

In [22]:
qingdan = qingdan_yuan.copy()

In [23]:
keyan_zdanzi = qingdan.loc[(qingdan['催可研']=='可研') ]['唯一编号'].count()
keyan_z = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
keyan_w = qingdan.loc[(
    qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [24]:
'催可研工单共'+str(keyan_zdanzi)+'单，涉及'+str(keyan_z)+'个逻辑站点工期催研，目前累\
计'+str(keyan_w)+'个站点已纳入工期，入研进度为'+str(keyan_w/keyan_z*100)[:5]+'%。'

'催可研工单共2081单，涉及1287个逻辑站点工期催研，目前累计785个站点已纳入工期，入研进度为60.99%。'

In [25]:
#可研总量和完成的df
keyan_quchong = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') 
keyan_quchong_wancheng = qingdan.loc[(
    qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') 

In [26]:
#个地市的进度
keyan_zongliang = keyan_quchong.groupby(by = '地市').agg({'唯一编号':'count'})
keyan_wanchengliang = keyan_quchong_wancheng.groupby(by = '地市').agg({'唯一编号':'count'})
keyanjindu=pd.concat([keyan_zongliang,keyan_wanchengliang],axis = 1)
keyanjindu.columns = ['总量','完成量']
keyanjindu['进度'] = keyanjindu['完成量']/keyanjindu['总量']

In [27]:
keyanjindu

,总量,完成量,进度
地市,,,
十堰,104,47,0.451923
咸宁,199,126,0.633166
天门,27,22,0.814815
孝感,89,55,0.617978
宜昌,80,14,0.175000
恩施,102,79,0.774510
武汉,111,101,0.909910
江汉,18,15,0.833333
潜江,6,3,0.500000


# 规划进度

In [28]:
qingdan = qingdan_yuan.copy()

In [29]:
guihua_zdanzi = qingdan.loc[(qingdan['规划']=='规划') ]['唯一编号'].count()
guihua_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]['唯一编号'].count()
'需规划解决工单共'+str(guihua_zdanzi)+'单，目前累计完成'+str(guihua_w)+'个问题工单规划衔接，规\
划完成比例'+str(guihua_w/guihua_zdanzi*100)[:5]+'%。'

'需规划解决工单共6492单，目前累计完成2660个问题工单规划衔接，规划完成比例40.97%。'

In [30]:
#规划总量和完成的df
guihua_quchong = qingdan.loc[(qingdan['规划']=='规划') ]
guihua_quchong_wancheng = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]

In [31]:
#个地市的进度
guihua_zongliang = guihua_quchong.groupby(by = '地市').agg({'唯一编号':'count'})
guihua_wanchengliang = guihua_quchong_wancheng.groupby(by = '地市').agg({'唯一编号':'count'})
guihuajindu=pd.concat([guihua_zongliang,guihua_wanchengliang],axis = 1)
guihuajindu.columns = ['总量','完成量']
guihuajindu['进度'] = guihuajindu['完成量']/guihuajindu['总量']

# ---------周进度----------

In [32]:
line = '''
1.针对2020年的MR竞对覆盖提升清单，总任务量共计12009个。
 规（小区）/优（小区）/研（站点）/建（站点）分别为6492/ 4313/ 1287/1760,
 本周新增分别为0/3/0/146，累计已完成2604（40.11%）/ 957（22.19%）/ 784（60.92%）/ 1027（58.35%）。
 '''

In [33]:
keyanjindu.columns = ['总量',  '完成量', '可研进度']
keyanjindu1=keyanjindu[['可研进度']]
cuikaijindu.columns = ['总量',  '完成量', '催开进度']
cuikaijindu1=cuikaijindu[['催开进度']]
youhuajindu.columns = ['总量', '派发量', '完成量','完成派发进度', '优化进度']
youhuajindu1=youhuajindu[['优化进度']]
guihuajindu.columns = ['总量', '完成量', '规划进度']
guihuajindu1=guihuajindu[['规划进度']]
data_z = pd.concat([youhuajindu1,cuikaijindu1,keyanjindu1,guihuajindu1],axis=1)

In [34]:
pattern = re.compile(r'.*累计已完成(\d*).*/ (\d*).*/ (\d*).*/ (\d*).*')                    # 用于匹配至少一个数字
m = pattern.search(line) 
g_s = m.group(1)
y_s = m.group(2)
k_s = m.group(3)
j_s = m.group(4)
gui = guihua_w-int(g_s)
you = youhua_w-int(y_s)
yan = keyan_w-int(k_s)
jian = cuikai_w-int(j_s)

In [35]:
gui,you,yan,jian

(56, 8, 1, 20)

In [36]:
print('1.针对2020年的MR竞对覆盖提升清单，总任务量共计12009个。\n \
规（小区）/优（小区）/研（站点）/建（站点）分别为{g}/ {y}/ {k}/{j},\n \
本周新增分别为{gui}/{you}/{yan}/{jian}，累计已完成{g_w}（{gui_b}%）/ {y_w}（{you_b}%）/ {ky_w}（{yan_b}%）/ {c_w}（{jian_b}%）。'.format(
    g_w=str(guihua_w),ky_w=str(keyan_w),c_w =str(cuikai_w),y_w = str(youhua_w) ,g=str(guihua_zdanzi),
    y=str(youhua_z),k=str(keyan_z),j=str(cuikai_z),gui=gui  ,you= you ,yan= yan ,jian = jian ,
    gui_b=round(guihua_w/guihua_zdanzi*100,2),
    you_b=round(youhua_w/youhua_z*100,2),
    yan_b=round(keyan_w/keyan_z*100,2),
    jian_b=round(cuikai_w/cuikai_z*100,2)
))

1.针对2020年的MR竞对覆盖提升清单，总任务量共计12009个。
 规（小区）/优（小区）/研（站点）/建（站点）分别为6492/ 4313/ 1287/1760,
 本周新增分别为56/8/1/20，累计已完成2660（40.97%）/ 965（22.37%）/ 785（60.99%）/ 1047（59.49%）。


# ------------武汉竞对及MR分析----------------
### 优化

In [33]:
qingdan_y = qingdan_yuan.copy()
qingdan_y = qingdan_y.loc[qingdan_y['地市']=='武汉']
qingdan = qingdan_y.loc[qingdan_y['地市']=='武汉']

In [34]:
qingdan.columns

Index([       '唯一编号', '问题点或问题小区cgi',       '对应cgi',         '小区名',
                7734,     '所属网洞ID_',     '问题点中心经度',     '问题点中心纬度',
             '是否计划内',          '分类',          '地市',          '区县',
              '区县有效',          '厂家',        '联通黑点',        '电信黑点',
              '是否黑点',          '场景', '9月增补场景-核心城区',   '9月增补场景-农村',
       '9月增补场景-交通枢纽',        '覆盖类型',       '小区名.1',          '总点',
                '弱点',         '覆盖率',       '联通覆盖率',       '改善后总点',
             '改善后弱点',      '改善后覆盖率',       '电信覆盖率',         'lon',
               'lat',        '天线挂高',         '方位角',      '覆盖类型.1',
              '工作频段',    '设备类型(大类)',        '区域类型',          '优化',
                '规划',         '催可研',          '催开',       '方案-最新',
        'TDL规划站号（新）',        '规划站名',          '站型',        '工期核实',
              '优化完成',        '规划完成',        '可研完成',        '催开完成',
             '优化已派发',      '优化派发时间',       '派发处方案',        '优化有效',
            '优化派发批次',          '备注',      '优化3500',    '7734不能

In [35]:
youhua_z = qingdan.loc[(qingdan['优化']=='优化')]['唯一编号'].count()
youhua_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]['唯一编号'].count()
youhua_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)& (qingdan['优化完成']==1)]['唯一编号'].count()

In [36]:
youhua_w

218

In [37]:
youhua_data_z = qingdan.loc[(qingdan['优化']=='优化') ]
youhua_data_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]
youhua_data_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1) & (qingdan['优化完成']==1)]

In [38]:
youhua_data_p.shape

(331, 78)

In [39]:
y_w = youhua_data_w.groupby(by = '区县').agg({'唯一编号':'count'})
y_p = youhua_data_p.groupby(by = '区县').agg({'唯一编号':'count'})
y_z = youhua_data_z.groupby(by = '区县').agg({'唯一编号':'count'})
youhuajindu=pd.concat([y_z,y_p,y_w],axis = 1)

In [40]:
youhuajindu.columns = ['总量','派发量','完成量']
youhuajindu['派发完成进度'] = youhuajindu['完成量']/youhuajindu['派发量']
youhuajindu['进度'] = youhuajindu['完成量']/youhuajindu['总量']

In [41]:
youhuajindu

,总量,派发量,完成量,派发完成进度,进度
区县,,,,,
东新,95,20,6,0.300000,0.063158
东西湖,48,30,25,0.833333,0.520833
新洲,154,17,14,0.823529,0.090909
武昌,35,13,8,0.615385,0.228571
汉阳,79,40,13,0.325000,0.164557
江夏,72,16,10,0.625000,0.138889
江岸,135,25,23,0.920000,0.170370
江汉,85,24,22,0.916667,0.258824
洪山,24,9,4,0.444444,0.166667


In [42]:
youhua_wanchengdugao=list(youhuajindu.sort_values(by="派发完成进度",ascending= False).head(3).index)  #排序 升序True 降序False by列

In [43]:
'MR竞对优化工单'+str(youhua_z)+'单。已派发'+str(youhua_p)+'已完成'+str(youhua_w)+'完成率为'+str(youhua_w/youhua_p*100)[:5]+'%'+str(
    youhua_wanchengdugao[0])+'，'+str(youhua_wanchengdugao[1])+'，'+str(youhua_wanchengdugao[2])+'派发工单解决率相对较高'

'MR竞对优化工单1077单。已派发331已完成218完成率为65.86%江岸，江汉，东西湖派发工单解决率相对较高'

In [44]:
youhuajindu.to_clipboard()

# 催开

In [45]:
qingdan_c = qingdan_yuan.copy()
qingdan_c = qingdan_c.loc[qingdan_c['地市']=='武汉']
qingdan = qingdan_c.copy()

In [46]:
cuikai_zdanzi = qingdan.loc[(qingdan['催开']=='催开') ]['唯一编号'].count()
cuikai_z = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
cuikai_w = qingdan.loc[(qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(
    subset='TDL规划站号（新）') ['唯一编号'].count()

In [47]:
cuikai_quchong = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') 
cuikai_quchong_wancheng = qingdan.loc[(qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(
    subset='TDL规划站号（新）') 

In [48]:
cuikai_zongliang = cuikai_quchong.groupby(by = '区县').agg({'唯一编号':'count'})
cuikai_wanchengliang = cuikai_quchong_wancheng.groupby(by = '区县').agg({'唯一编号':'count'})

In [49]:
cuikaijindu=pd.concat([cuikai_zongliang,cuikai_wanchengliang],axis = 1)
cuikaijindu.columns = ['总量','完成量']
cuikaijindu['进度'] = cuikaijindu['完成量']/cuikaijindu['总量']

In [50]:
cuikai_wanchengdudi=list(cuikaijindu.sort_values(by="进度",ascending= True).head(1).index)  #排序 升序True 降序False by列

a=cuikai_wanchengdudi[0]

jindudi=cuikai_quchong.loc[(cuikai_quchong['区县']==a) & (cuikai_quchong['催开完成']!=1)].groupby(by='工期核实').agg(
    {'唯一编号':'count'}).sort_values(by="唯一编号",ascending= False).head(1).index

In [51]:
a

'新洲'

In [52]:
'催开工单共'+str(cuikai_zdanzi)+'单，涉及逻辑站\
点'+str(cuikai_z)+'个，已开通站点'+str(cuikai_w)+'个(本月新增201个），催开完成率\
为'+str(cuikai_w/cuikai_z*100)[:5]+'%'+'。'+cuikai_wanchengdudi[0]+'地市整体解决进度较慢，主要为'+jindudi[0]+'未开通占比较\
高，需加快站点建设进度。'+'已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

'催开工单共881单，涉及逻辑站点550个，已开通站点401个(本月新增201个），催开完成率为72.90%。新洲地市整体解决进度较慢，主要为5月前完成-2020年湖北无线网4G主设备扩容工程第一批未开通占比较高，需加快站点建设进度。已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

In [53]:
cuikaijindu

,总量,完成量,进度
区县,,,
东新,86,62,0.720930
东西湖,19,15,0.789474
新洲,42,12,0.285714
武昌,19,16,0.842105
汉阳,39,34,0.871795
江夏,65,52,0.800000
江岸,45,42,0.933333
江汉,43,34,0.790698
洪山,20,16,0.800000


In [54]:
cuikaijindu.to_clipboard()

# 可研

In [ ]:
qingdan_k = qingdan_yuan.copy()
qingdan_k = qingdan_k.loc[qingdan_k['地市']=='武汉']
qingdan = qingdan_k.copy()

In [ ]:
keyan_zdanzi = qingdan.loc[(qingdan['催可研']=='可研') ]['唯一编号'].count()
keyan_z = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
keyan_w = qingdan.loc[(qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(
    subset='TDL规划站号（新）') ['唯一编号'].count()

In [ ]:
'催可研工单共'+str(keyan_zdanzi)+'单，涉及'+str(keyan_z)+'个逻辑站点工期催研，目前累\
计'+str(keyan_w)+'个站点已纳入工期，入研进度为'+str(keyan_w/keyan_z*100)[:5]+'%。'

In [ ]:
keyan_quchong = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）')
keyan_quchong_w = qingdan.loc[(qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）')

In [ ]:
k_z = keyan_quchong.groupby(by = '区县').agg({'唯一编号':'count'})
k_w = keyan_quchong_w.groupby(by = '区县').agg({'唯一编号':'count'})
keyanjindu=pd.concat([k_z,k_w],axis = 1)
keyanjindu.columns = ['总量','完成量']
keyanjindu['进度'] = keyanjindu['完成量']/keyanjindu['总量']

In [ ]:
keyanjindu.to_clipboard()

# 规划 

In [ ]:
qingdan_g = qingdan_yuan.copy()
qingdan_g = qingdan_g.loc[qingdan_g['地市']=='武汉']
qingdan = qingdan_g.copy()

guihua_zdanzi = qingdan.loc[(qingdan['规划']=='规划') ]['唯一编号'].count()
guihua_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]['唯一编号'].count()

In [ ]:
'需规划解决工单共'+str(guihua_zdanzi)+'单，目前累计完成'+str(guihua_w)+'个问题工单规划衔接，规\
划完成比例'+str(guihua_w/guihua_zdanzi*100)[:5]+'%。'

In [ ]:
guihua_qingdan = qingdan.loc[(qingdan['规划']=='规划') ]
guihua_qingdan_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1)]
g_z = guihua_qingdan.groupby(by = '区县').agg({'唯一编号':'count'})
g_w = guihua_qingdan_w.groupby(by = '区县').agg({'唯一编号':'count'})
guihuajindu=pd.concat([g_z,g_w],axis = 1)
guihuajindu.columns = ['总量','完成量']
guihuajindu['进度'] = guihuajindu['完成量']/guihuajindu['总量']

In [ ]:
guihuajindu.to_clipboard()